In [1]:
from fastai.text.all import *
import gc

In [2]:
torch.cuda.is_available()

True

#### Fine-tuning a language model on Amazon Dataset Reviews
Data source: https://nijianmo.github.io/amazon/index.html

In [3]:
from IPython.display import Image
Image(url='https://docs.fast.ai/images/ulmfit.png')

In [4]:
path = Path("..") / "data/sample/"

In [5]:
df = pd.read_csv(path/"data.csv")
sample_df = df.sample(frac=0.02) #forgot to set initial seed, will set before submiting

In [6]:
sample_df.shape

(160988, 1)

In [6]:
del df
gc.collect()

1406

In [7]:
BATCH_SIZE = 16
dls_lm = TextDataLoaders.from_df(sample_df, path=path, text_col=0, label_col=1, is_lm=True, bs=BATCH_SIZE, valid_pct=0.1)

/home/lazylearner/anaconda3/envs/amazonreviews/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [8]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj my wife dreams out loud of one republic , but not of me . xxmaj thanks man . xxmaj you stole my girl . \n\n xxmaj good xxmaj album . xxbos xxmaj an unabashed xxmaj kinks fan for a long time now , i wondered when an enterprising producer would mine the xxmaj kinks xxunk for one of these tribute compilations . xxmaj jim xxmaj pitt 's xxmaj this xxmaj","xxmaj my wife dreams out loud of one republic , but not of me . xxmaj thanks man . xxmaj you stole my girl . \n\n xxmaj good xxmaj album . xxbos xxmaj an unabashed xxmaj kinks fan for a long time now , i wondered when an enterprising producer would mine the xxmaj kinks xxunk for one of these tribute compilations . xxmaj jim xxmaj pitt 's xxmaj this xxmaj is"
1,"and beauty . xxmaj it is a pleasure to write with them and they are suitable for all fountain pens . i bought a dozen of different colors . xxmaj this is a wonderful green - blue color , very subtle and elegant . xxmaj the bottle is superb . xxmaj using such an ink , everyone could personalize his / her writing . xxbos xxmaj absolutely great cable charger . i","beauty . xxmaj it is a pleasure to write with them and they are suitable for all fountain pens . i bought a dozen of different colors . xxmaj this is a wonderful green - blue color , very subtle and elegant . xxmaj the bottle is superb . xxmaj using such an ink , everyone could personalize his / her writing . xxbos xxmaj absolutely great cable charger . i love"
2,"your head . "" xxunk y xxmaj sigue "" is a lovely salsa influenced track which again is so catchy will drive you nuts . xxmaj but my favourite track is the perfectly executed "" xxunk a xxmaj poco "" , xxmaj estopa 's view on smoking . xxmaj taking from the point of a smoker it describes his feelings and why he does it and what it is doing it to","head . "" xxunk y xxmaj sigue "" is a lovely salsa influenced track which again is so catchy will drive you nuts . xxmaj but my favourite track is the perfectly executed "" xxunk a xxmaj poco "" , xxmaj estopa 's view on smoking . xxmaj taking from the point of a smoker it describes his feelings and why he does it and what it is doing it to him"
3,"xxmaj oddly , the stations seemed to play "" dogs "" now and then , even though it was not released as a single . "" give a xxmaj hand , xxmaj take a xxmaj hand "" was an older song from 1969 that the xxmaj gibbs rerecorded after xxmaj the xxmaj staple xxmaj singers did well with it . xxmaj the song styles waver through soft , sensual ballads like ""","oddly , the stations seemed to play "" dogs "" now and then , even though it was not released as a single . "" give a xxmaj hand , xxmaj take a xxmaj hand "" was an older song from 1969 that the xxmaj gibbs rerecorded after xxmaj the xxmaj staple xxmaj singers did well with it . xxmaj the song styles waver through soft , sensual ballads like "" xxunk"
4,end 1996 xxup f250 xxbos xxmaj it works . xxbos xxmaj the only solid white leather strap i could find that did n't have any ornamentation . xxmaj it is of average quality but for the price i was n't expecting much . xxmaj it looks nice and is mostly comfortable on my black xxmaj ibanez xxup xxunk . xxbos very good xxbos i went to the local premiere of xxmaj mary,1996 xxup f250 xxbos xxmaj it works . xxbos xxmaj the only solid white leather strap i could find that did n't have any ornamentation . xxmaj it is of average quality but for the price i was n't expecting much . xxmaj it looks nice and is mostly comfortable on my black xxmaj ibanez xxup xxunk . xxbos very good xxbos i went to the local premiere of xxmaj mary xxmaj


In [9]:
dls_lm.bs

16

In [10]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [11]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(43600, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(43600, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=43600, bias=True)
    (output_dp): RNNDropout()
  )
)

#### Train only the head of the model while the body stays frozen.

In [14]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.463609,4.205848,0.272985,67.077438,34:35


In [15]:
learn.save('1epoch-amazon-reviews-sample')

Path('../data/sample/models/1epoch-amazon-reviews-sample.pth')

In [16]:
# learn = learn.load('1epoch')

#### Fine-tune the model after unfreezing

In [16]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.091950,3.965241,0.294048,52.732975,1:27:53
1,4.138435,4.004159,0.290700,54.825722,1:50:32
2,4.075377,3.994715,0.291748,54.310360,2:00:35
3,4.099702,3.969203,0.294860,52.942307,1:47:00
4,3.998508,3.940119,0.298114,51.424698,1:47:01
5,4.016881,3.898405,0.302949,49.323696,1:47:03
6,3.918993,3.859989,0.307038,47.464836,1:47:01
7,3.890229,3.825045,0.311247,45.834869,1:47:17
8,3.837510,3.801839,0.314293,44.783470,1:52:52
9,3.823071,3.796320,0.315118,44.537006,1:48:25


In [17]:
learn.save_encoder('finetuned-amazon-reviews-sample')

In [18]:
TEXT = "Good product"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [19]:
print("\n".join(preds))

Good product . Easy to install . Works with removing the battery . This is a very good selection of Prince arrangements . The vocals are pretty good . This is my second
Good product . Fast shipping . Great price . i have used this LG LG G4 Screen Protector for a month but it is a great phone . The only problem i have


In [20]:
TEXT = "Worst delivery"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [21]:
print("\n".join(preds))

Worst delivery ever . Wo n't buy again . Good product ! ZERO STARS Well , i bought this case for my daughter who has the apple iphone + 6s Plus , and she
Worst delivery i could find from Amazon . The Top 5 masterpiece is Prince 's The 70 's Landscapes , One Day of Life , and the Rock By


In [22]:
TEXT = "Poor service"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [23]:
print("\n".join(preds))

Poor service , good price , came from China , very disappointed . It 's very small , but it 's somewhat small on the side and is very short so i have to pull it over and over
Poor service and product This is a very nice case . It 's not too bulky but it 's very durable . i use it for my iphone 5 and it does n't worry when the phone is in


In [36]:
TEXT = "Looked good in pictures, fabric is also great"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.3) 
         for _ in range(N_SENTENCES)]

In [37]:
print("\n".join(preds))

Looked good in pictures , fabric is also great . It 's a little bit thicker than i thought it would be . But it 's a good quality , and i am very happy with it . This is a great product .
Looked good in pictures , fabric is also great . The color is a little darker than the picture , but it 's a little more bright than the picture . It 's a nice color , but i have n't had any problems with it .


In [38]:
TEXT = "Not satisfied with the product"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.3) 
         for _ in range(N_SENTENCES)]

In [39]:
print("\n".join(preds))

Not satisfied with the product . The product was not as described . This is a great product . It is an amazing product . It is easy to use and it is easy to use . The only
Not satisfied with the product . It is not a bad product . It is very difficult to put on and off . It does not fit the phone . It is not the case that does not fit my phone


In [41]:
TEXT = "This mobile cover"
N_WORDS = 30
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [42]:
print("\n".join(preds))

This mobile cover is either a little flimsy and a little pricey . It is much lighter than a standard sprayer . It feels durable and is easy to use .
This mobile cover is very easy to assemble . It is simple , well made and requires a little effort to assemble . It works well , with no effort at
This mobile cover works great for my small backyard of home . i have several large suburban - room areas that stick into my house and i have a lot of clippings and
This mobile cover is perfect for our Home Depot store . It is manufactured for Christmas and has held up well . i expect it to last a long
This mobile cover for the tailgating , camping , camping trips and camping trips , is easy to manage , and it is easy to travel with a steak or grill .


In [47]:
TEXT = "Different product received"
N_WORDS = 30
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [48]:
print("\n".join(preds))

Different product received in my opinion . Fits very well . Really like it so far . Works great , needs to be cleaned , and does n't leak
Different product received . Not the one i was expecting . The second one was shipped immediately . The price was the worst price on the lower priced spools at
Different product received my Black & Decker Edge Hog . It worked well and was easy to use . i use it to trim my shrubs and trees
Different product received . This is my first bug zapper and I 've installed it on the Flowtron brand unit . It kills bugs , even with the
Different product received by Amazon for a great product . These are pretty good . i really like them , just how they last . We got this


In [52]:
TEXT = "This has helped me so much"
N_WORDS = 20
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.7) 
         for _ in range(N_SENTENCES)]

In [53]:
print("\n".join(preds))

This has helped me so much . It is made of a good quality quality and the price is good . This is
This has helped me so much better . i am a huge fan of Richard Wrist . It is one of my favorite


In [56]:
TEXT = "Bad product"
N_WORDS = 8
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.2) 
         for _ in range(N_SENTENCES)]

print("\n".join(preds))

Bad product . It is a little pricey .
Bad product . It is not a good product
Bad product . The product is not as described
Bad product . It is a little too small
Bad product . Not very good .
